# Bi-Directional LSTM

In [20]:
from keras.layers import Input, Conv1D, MaxPooling1D, Bidirectional, LSTM, concatenate, Dense, GlobalAveragePooling1D, GlobalMaxPooling1D

import pandas as pd
import matplotlib.pyplot as plt
# load the data
df = pd.read_csv('3polarity.csv', encoding = 'utf-8')

In [21]:
from sklearn.metrics import classification_report

def print_classification_report(model, X_test, y_test):
    pred = model.predict(X_test)
    pred_classes = np.argmax(pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)

    print(classification_report(y_test_classes, pred_classes))
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print('Test loss:', loss)
    print('Test accuracy:', accuracy)


In [31]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Embedding, Dropout, Conv1D, MaxPooling1D, Flatten, Dense
from keras.models import Model
from sklearn.model_selection import train_test_split


def preprocess_data(df):
    tokenizer = Tokenizer(num_words=5000, split=' ')
    tokenizer.fit_on_texts(df['Tweet'].values)
    X = tokenizer.texts_to_sequences(df['Tweet'].values)
    X = pad_sequences(X)
    y = pd.get_dummies(df['Polarity']).values
    return X, y


def build_bilstm_model(input_shape):
    inputs = Input(shape=input_shape)
    x = Embedding(5000, 128, input_length=input_shape[0])(inputs)
    x = Dropout(0.5)(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Conv1D(64, kernel_size=3, padding="valid", kernel_initializer="glorot_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool])
    x = Dense(64, activation='relu')(x)  # add one more dense layer
    x = Dropout(0.5)(x)
    x = Dense(32, activation='relu')(x)  # add one more dense layer
    x = Dropout(0.5)(x)
    outputs = Dense(3, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def train_bilstm_model(df):
    X, y = preprocess_data(df)
    input_shape = X.shape[1:]
    model = build_bilstm_model(input_shape)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=128)
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print('Test loss:', loss)
    print('Test accuracy:', accuracy)
    return model, history


In [32]:
bilstm_model,bilstm_history = train_bilstm_model(df)

Epoch 1/10
37/37 [==============================] - 15s 290ms/step - loss: 0.9851 - accuracy: 0.5296 - val_loss: 0.9352 - val_accuracy: 0.5346
Epoch 2/10
37/37 [==============================] - 10s 252ms/step - loss: 0.8219 - accuracy: 0.6456 - val_loss: 0.7662 - val_accuracy: 0.6852
Epoch 3/10
37/37 [==============================] - 10s 272ms/step - loss: 0.6349 - accuracy: 0.7438 - val_loss: 0.6696 - val_accuracy: 0.7160
Epoch 4/10
37/37 [==============================] - 11s 310ms/step - loss: 0.4534 - accuracy: 0.8093 - val_loss: 0.6378 - val_accuracy: 0.7297
Epoch 5/10
37/37 [==============================] - 11s 286ms/step - loss: 0.3207 - accuracy: 0.8481 - val_loss: 0.7374 - val_accuracy: 0.7177
Epoch 6/10
37/37 [==============================] - 9s 247ms/step - loss: 0.2609 - accuracy: 0.8729 - val_loss: 0.7840 - val_accuracy: 0.7169
Epoch 7/10
37/37 [==============================] - 9s 245ms/step - loss: 0.2393 - accuracy: 0.8817 - val_loss: 0.8619 - val_accuracy: 0.7228
E